# 目的
OpenMath2-Llama3.1-8Bに対して、apply_chat_templateを使うテスト

ref: 
- https://github.com/unslothai/unsloth
- https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing#scrollTo=6bZsfBuZDeCL
- https://huggingface.co/blog/mlabonne/sft-llama3
- https://huggingface.co/docs/trl/sft_trainer#accelerate-fine-tuning-2x-using-unsloth

In [1]:
# pip install "unsloth[cu121-torch230] @ git+https://github.com/unslothai/unsloth.git"
# -> だめ

# pip uninstall torch
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# pip install "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"
# -> OK

In [2]:
# path setting
EXP_NAME = "e018-causal"
# MODEL_NAME = "unsloth/gemma-2-9b-it-bnb-4bit"
# MODEL_NAME = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
# MODEL_NAME = "nvidia/OpenMath2-Llama3.1-8B"  # OOMになってしまう
MODEL_NAME = "QuantFactory/OpenMath2-Llama3.1-8B-GGUF"
COMPETITION_NAME = "eedi-mining-misconceptions-in-mathematics"

DATA_PATH = "data"
DATASET_NAME = f"{EXP_NAME}-{MODEL_NAME.split('/')[-1].replace('.', '-')}"
OUTPUT_PATH = f"output/causal/{EXP_NAME}"
MODEL_OUTPUT_PATH = f"{OUTPUT_PATH}/trained_model"
ENV_PATH = "env_file"

# experiment parameter
DEBUG = False
TRAINING = True
UPLOAD_DATA_TO_S3 = True
UPLOAD_DATA_TO_KAGGLE = True
REMOVE_LOCAL_FILE = False
WANDB = True

# USE_FOLD = 0
# USE_DATA_RATE = 1.0
# VALID_DATA_SIZE = 3000

# model parameter
TRAINING_MAX_LENGTH = 512
SEED = 42
EPOCH = 5
LR = 2e-04
TRAIN_BS = 1  # 2  # 4  # 16
GRAD_ACC_STEP = 128 // TRAIN_BS  # 仮想的なバッチサイズはTRAIN_BS * GRAD_ACC_STEPとなる
EVAL_BS = 1  # 2  # 4  # 16
# NUM_LABELS = 3

FREEZE_LAYERS = 24  # 16

# rola parameter
LORA_R = 8  # 16
LORA_ALPHA = LORA_R * 2
LORA_BIAS = "none"

RESUME_FROM_CHECKPOINT = False  # 途中から再開する場合はTrueにする

In [3]:
!nvidia-smi

Fri Oct 18 02:41:30 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090 Ti     On  |   00000000:05:00.0 Off |                  Off |
| 35%   58C    P5             38W /  350W |       2MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
!python --version

Python 3.10.14


In [5]:
def resolve_path(base_path: str) -> str:
    import os

    cwd = os.getcwd()
    print(cwd)
    if cwd == f"/notebooks":
        print("Jupyter Kernel By VSCode!")
        return f"/notebooks/{COMPETITION_NAME}/{base_path}"
    elif cwd == f"/notebooks/{COMPETITION_NAME}":
        print("nohup!")
        return base_path
    elif cwd == f"/notebooks/{COMPETITION_NAME}/{COMPETITION_NAME}/exp":
        print("Jupyter Lab!")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/reranker":
        print("VastAi! Reranker")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/retriever":
        print("VastAi! Retriever")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/causal":
        print("VastAi! Causal")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}":
        print("VastAi!")
        return base_path
    else:
        raise Exception("Unknown environment")


DATA_PATH = resolve_path(DATA_PATH)
print(DATA_PATH)
OUTPUT_PATH = resolve_path(OUTPUT_PATH)
print(OUTPUT_PATH)
MODEL_OUTPUT_PATH = resolve_path(MODEL_OUTPUT_PATH)
print(MODEL_OUTPUT_PATH)
ENV_PATH = resolve_path(ENV_PATH)
print(ENV_PATH)

/root/eedi-mining-misconceptions-in-mathematics/exp/causal
VastAi! Causal
../../data
/root/eedi-mining-misconceptions-in-mathematics/exp/causal
VastAi! Causal
../../output/causal/e018-causal
/root/eedi-mining-misconceptions-in-mathematics/exp/causal
VastAi! Causal
../../output/causal/e018-causal/trained_model
/root/eedi-mining-misconceptions-in-mathematics/exp/causal
VastAi! Causal
../../env_file


In [6]:
def validate_dataset_name(dataset_name: str) -> None:
    if len(dataset_name) < 6 or len(dataset_name) > 50:
        raise Exception(
            f"データセットの文字列は6~50文字にしてください。現在{len(DATASET_NAME)}文字"
        )
    if "_" in dataset_name or "." in dataset_name:
        raise Exception("datasetの名称に'_'、'.'の使用は禁止です")


validate_dataset_name(DATASET_NAME)

# import

In [7]:
import os
import random

import polars as pl
import numpy as np

import wandb

import torch
from datasets import Dataset
from transformers import TrainingArguments

from trl import SFTTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [8]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
NUM_PROC = 16  # os.cpu_count()

In [9]:
import importlib.metadata
import torch
import transformers
import datasets
import trl

assert torch.__version__ == "2.4.1+cu121"
assert transformers.__version__ == "4.44.2"
assert datasets.__version__ == "3.0.0"
assert trl.__version__ == "0.11.1"
assert importlib.metadata.version("unsloth") == "2024.10.1"

In [10]:
# Seed the same seed to all
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything(SEED)

In [11]:
from dotenv import load_dotenv

load_dotenv(f"{ENV_PATH}/.env")

True

# Wandb

In [12]:
if WANDB:
    wandb.login(key=os.environ["WANDB_API_KEY"])
    wandb.init(project=COMPETITION_NAME, name=EXP_NAME)
    REPORT_TO = "wandb"
else:
    REPORT_TO = "none"

REPORT_TO

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sinchir0. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


'wandb'

# Model Tokenizer Load

In [13]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=TRAINING_MAX_LENGTH,
    # dtype=None, # 学習のタイミングでRuntimeError: Expected out tensor to have dtype c10::Half, but got float insteadと出てしまう
    dtype=torch.float16,
    # load_in_4bit=True,
)

==((====))==  Unsloth 2024.10.1: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3090 Ti. Max memory: 23.67 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

nvidia/OpenMath2-Llama3.1-8B does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


In [14]:
# https://www.kaggle.com/code/kishanvavdara/lmsys-llama-3-tpu-train
# tokenizer.padding_side = "right"

# Data Import & Preprocess

In [15]:
train_long = pl.read_csv(f"{DATA_PATH}/train_long.csv").filter(
    (pl.col("CorrectAnswer") != pl.col("AnswerAlphabet"))
    & (pl.col("MisconceptionId").is_not_null())
)

In [16]:
if DEBUG:
    train_long = train_long.head(100)

In [17]:
# messages = [
#     {
#         "role": "system",
#         "content": "You are a mathematics expert. You are given a mathematics question, an incorrect answer, a correct answer, a construct name, and a subject name. Using these, identify what misconception exists in the incorrect answer.",
#     },
#     {"role": "user", "content": "Who are you?"},
# ]

# # TODO: CoT(think step by step)をどうFine-Tuningに反映させるべきかを調べる
# alpaca_prompt = """### Input:
# Question: {question_text}
# Incorrect Answer: {incorrect_answer_text}
# Correct Answer: {correct_answer_text}
# Construct Name: {construct_name}
# Subject Name: {subject_name}

# ### Response:
# {misconception_name}"""

# EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


# def formatting_prompts_func(examples):
#     texts = []
#     for (
#         question_text,
#         incorrect_answer_text,
#         correct_answer_text,
#         construct_name,
#         subject_name,
#         misconception_name,
#     ) in zip(
#         examples["QuestionText"],
#         examples["AnswerText"],  # incorrect_answer_text
#         examples["CorrectAnswerText"],
#         examples["ConstructName"],
#         examples["SubjectName"],
#         examples["MisconceptionName"],
#     ):
#         # Must add EOS_TOKEN, otherwise your generation will go on forever!
#         text = (
#             alpaca_prompt.format(
#                 question_text=question_text,
#                 incorrect_answer_text=incorrect_answer_text,
#                 correct_answer_text=correct_answer_text,
#                 construct_name=construct_name,
#                 subject_name=subject_name,
#                 misconception_name=misconception_name,
#             )
#             + EOS_TOKEN
#         )
#         texts.append(text)
#     return {
#         "text": texts,
#     }


# train = Dataset.from_polars(train_long).map(
#     formatting_prompts_func,
#     batched=True,
# )

In [18]:
QUESTION_TEMPLATE = """Question: {question_text}
Incorrect Answer: {incorrect_answer_text}
Correct Answer: {correct_answer_text}
Construct Name: {construct_name}
Subject Name: {subject_name}"""


class ApplyTemplateForTrain:
    def make_masaages(self, question: str, answer: str) -> list[dict[str, str]]:
        return [
            {
                "role": "system",
                "content": "You are a mathematics expert. You are given a mathematics question, an incorrect answer, a correct answer, a construct name, and a subject name. Using these, identify what misconception exists in the incorrect answer.",
            },
            {"role": "user", "content": question},
            {"role": "assistant", "content": answer},
        ]

    def apply_template(self, examples):
        messages = []
        for (
            question_text,
            incorrect_answer_text,
            correct_answer_text,
            construct_name,
            subject_name,
            misconception_name,
        ) in zip(
            examples["QuestionText"],
            examples["AnswerText"],  # incorrect_answer_text
            examples["CorrectAnswerText"],
            examples["ConstructName"],
            examples["SubjectName"],
            examples["MisconceptionName"],
        ):
            message = self.make_masaages(
                question=QUESTION_TEMPLATE.format(
                    question_text=question_text,
                    incorrect_answer_text=incorrect_answer_text,
                    correct_answer_text=correct_answer_text,
                    construct_name=construct_name,
                    subject_name=subject_name,
                ),
                answer=misconception_name,
            )
            messages.append(message)

        text = [
            tokenizer.apply_chat_template(
                message, tokenize=False, add_generation_prompt=False
            )
            for message in messages
        ]

        return {"text": text}


train = Dataset.from_polars(train_long).map(
    ApplyTemplateForTrain().apply_template, batched=True
)

Map:   0%|          | 0/4370 [00:00<?, ? examples/s]

In [19]:
print(train[0]["text"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a mathematics expert. You are given a mathematics question, an incorrect answer, a correct answer, a construct name, and a subject name. Using these, identify what misconception exists in the incorrect answer.<|eot_id|><|start_header_id|>user<|end_header_id|>

Question: \[
3 \times 2+4-5
\]
Where do the brackets need to go to make the answer equal \( 13 \) ?
Incorrect Answer: Does not need brackets
Correct Answer: \( 3 \times(2+4)-5 \)
Construct Name: Use the order of operations to carry out calculations involving powers
Subject Name: BIDMAS<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Confuses the order of operations, believes addition comes before multiplication<|eot_id|>


In [20]:
print(train[10]["text"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a mathematics expert. You are given a mathematics question, an incorrect answer, a correct answer, a construct name, and a subject name. Using these, identify what misconception exists in the incorrect answer.<|eot_id|><|start_header_id|>user<|end_header_id|>

Question: This pictogram shows the different types of music Bob has in his music collection.

How many Blues CDs does Bob have? ![A pictogram showing the number of CDs Bob has in his musical collection. Pop has 3 and a half symbols, rock has 2 symbols, blues has 2 and a quarter symbols, jazz has 3 and a quarter symbols and classical has 1 and three-quarter symbols. Each symbol represents 4 CDs.]()
Incorrect Answer: \( 2.25 \)
Correct Answer: \( 9 \)
Construct Name: Interpret a pictogram involving fractions of symbols
Subject Name: Pictogram<|eot_id|><|start_header_id|>assistant<|end_header_id|>

When interpreting a pictogram, thinks each symbol stands for 1<|eot

# Split Train Valid

In [21]:
train, valid = (
    train.filter(lambda x: x["QuestionId"] % 3 != 0, num_proc=NUM_PROC),
    train.filter(lambda x: x["QuestionId"] % 3 == 0, num_proc=NUM_PROC),
)

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Filter (num_proc=16):   0%|          | 0/4370 [00:00<?, ? examples/s]

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Filter (num_proc=16):   0%|          | 0/4370 [00:00<?, ? examples/s]

# Model Predict Check Before Training

In [22]:
print(" === Model Predict Check Before Training Start ===")

 === Model Predict Check Before Training Start ===


In [23]:
# EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


# def formatting_prompts_func_predict(examples):
#     texts = []
#     for (
#         question_text,
#         incorrect_answer_text,
#         correct_answer_text,
#         construct_name,
#         subject_name,
#     ) in zip(
#         examples["QuestionText"],
#         examples["AnswerText"],  # incorrect_answer_text
#         examples["CorrectAnswerText"],
#         examples["ConstructName"],
#         examples["SubjectName"],
#     ):
#         # Must add EOS_TOKEN, otherwise your generation will go on forever!
#         text = (
#             alpaca_prompt.format(
#                 question_text=question_text,
#                 incorrect_answer_text=incorrect_answer_text,
#                 correct_answer_text=correct_answer_text,
#                 construct_name=construct_name,
#                 subject_name=subject_name,
#                 misconception_name="",
#             )
#             + EOS_TOKEN
#         )
#         texts.append(text)
#     return {
#         "text": texts,
#     }


# valid_for_predict = valid.map(
#     formatting_prompts_func_predict,
#     batched=True,
# )

In [24]:
class ApplyTemplateForPredict:
    def make_masaages(self, question: str) -> list[dict[str, str]]:
        return [
            {
                "role": "system",
                "content": "You are a mathematics expert. You are given a mathematics question, an incorrect answer, a correct answer, a construct name, and a subject name. Using these, identify what misconception exists in the incorrect answer.",
            },
            {"role": "user", "content": question},
        ]

    def apply_template(self, examples):
        messages = []
        for (
            question_text,
            incorrect_answer_text,
            correct_answer_text,
            construct_name,
            subject_name,
        ) in zip(
            examples["QuestionText"],
            examples["AnswerText"],  # incorrect_answer_text
            examples["CorrectAnswerText"],
            examples["ConstructName"],
            examples["SubjectName"],
        ):
            message = self.make_masaages(
                question=QUESTION_TEMPLATE.format(
                    question_text=question_text,
                    incorrect_answer_text=incorrect_answer_text,
                    correct_answer_text=correct_answer_text,
                    construct_name=construct_name,
                    subject_name=subject_name,
                )
            )
            messages.append(message)

        text = [
            tokenizer.apply_chat_template(
                message, tokenize=False, add_generation_prompt=True
            )
            for message in messages
        ]

        return {"text": text}


valid_for_predict = valid.map(ApplyTemplateForPredict().apply_template, batched=True)

Map:   0%|          | 0/1460 [00:00<?, ? examples/s]

In [25]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference


def predict(valid: Dataset, idx: int, model):
    inputs = tokenizer(
        valid["text"][idx],
        return_tensors="pt",
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=64, do_sample=True)
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])
    print("\n### Answer Misconception Name:")
    print(valid["MisconceptionName"][idx])

In [26]:
predict(valid_for_predict, 0, model)

In [27]:
predict(valid_for_predict, 1, model)

In [28]:
predict(valid_for_predict, 2, model)

In [29]:
print(" === Model Predict Check Before Training End ===")

 === Model Predict Check Before Training End ===


# Model

In [30]:
# from peft import prepare_model_for_int8_training
# model = prepare_model_for_int8_training(model)
# 今の状態で動かなかれば、これを試す。

In [31]:
model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_R,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        # "o_proj",
        # "gate_proj",
        # "up_proj",
        # "down_proj",
        "embed_tokens",
        "lm_head",
    ],
    lora_alpha=LORA_ALPHA,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=SEED,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
    layers_to_transform=[i for i in range(32) if i >= FREEZE_LAYERS],
)

Unsloth: Offloading input_embeddings to disk to save VRAM


/opt/conda/lib/python3.10/site-packages/unsloth/models/_utils.py:887: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  offloaded_W = torch.load(filename, map_location = "cpu", 

Unsloth: Offloading output_embeddings to disk to save VRAM


Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2024.10.1 patched 32 layers with 8 QKV layers, 0 O layers and 0 MLP layers.


Unsloth: Casting embed_tokens to float32
Unsloth: Casting lm_head to float32


In [32]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): ModulesToSaveWrapper(
          (original_module): Embedding(128256, 4096)
          (modules_to_save): ModuleDict(
            (default): Embedding(128256, 4096)
          )
        )
        (layers): ModuleList(
          (0-23): 24 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
              (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
              (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (rotary_emb): LlamaExtendedRotaryEmbedding()
            )
            (mlp): LlamaMLP(
              (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
              (up_proj): Linear4bit(in_featur

In [33]:
training_arguments = TrainingArguments(
    output_dir=MODEL_OUTPUT_PATH,
    learning_rate=LR,
    per_device_train_batch_size=TRAIN_BS,
    gradient_accumulation_steps=GRAD_ACC_STEP,
    eval_accumulation_steps=GRAD_ACC_STEP,
    per_device_eval_batch_size=EVAL_BS,
    num_train_epochs=EPOCH,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=0.1,
    save_strategy="steps",
    save_steps=0.1,
    save_total_limit=1,
    logging_steps=2,
    seed=SEED,
    metric_for_best_model="eval_loss",
    warmup_ratio=0.1,
    lr_scheduler_type="cosine_with_restarts",
    report_to=REPORT_TO,
    run_name=EXP_NAME,
    load_best_model_at_end=True,
    fp16=True,
    fp16_full_eval=True,
    gradient_checkpointing=True,
)

In [34]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train,
    eval_dataset=valid,
    dataset_text_field="text",
    max_seq_length=TRAINING_MAX_LENGTH,
    dataset_num_proc=NUM_PROC,
    packing=False,  # Can make training 5x faster for short sequences.
    args=training_arguments,
)

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Map (num_proc=16):   0%|          | 0/2910 [00:00<?, ? examples/s]

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Map (num_proc=16):   0%|          | 0/1460 [00:00<?, ? examples/s]

In [35]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3090 Ti. Max memory = 23.67 GB.
8.859 GB of memory reserved.


In [36]:
if TRAINING:
    trainer_stats = trainer.train()
else:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=MODEL_OUTPUT_PATH + "/lora_model",
        max_seq_length=TRAINING_MAX_LENGTH,
        dtype=None,  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
        # load_in_4bit=True,  # Use 4bit quantization to reduce memory usage. Can be False.
    )

Counting untrained tokens:   0%|          | 0/2910 [00:00<?, ? examples/s]

Unsloth: Setting embed_tokens & lm_head untrained tokens to mean(trained) to counteract NaNs during training.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,910 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 128
\        /    Total batch size = 128 | Total steps = 110
 "-____-"     Number of trainable parameters = 1,051,852,800


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.96 GiB. GPU 0 has a total capacity of 23.67 GiB of which 1.71 GiB is free. Process 2671289 has 21.95 GiB memory in use. Of the allocated memory 21.52 GiB is allocated by PyTorch, and 112.37 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Predict Valid

In [ ]:
print(" === Model Predict Check After Training Start ===")

In [42]:
predict(valid_for_predict, 0, model)

In [43]:
predict(valid_for_predict, 1, model)

In [44]:
predict(valid_for_predict, 2, model)

In [ ]:
print(" === Model Predict Check After Training End ===")

# Save

In [ ]:
model.save_pretrained(f"{MODEL_OUTPUT_PATH}/lora_model")  # Local saving
tokenizer.save_pretrained(f"{MODEL_OUTPUT_PATH}/lora_model")

# AWSへのアップロード

In [46]:
# S3へのアップロード
if not DEBUG and UPLOAD_DATA_TO_S3:
    # uninstall
    !sudo rm /usr/bin/aws
    !sudo rm /usr/bin/aws_completer
    !sudo rm -rf /usr/local/aws-cli

    # install
    !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
    !unzip -o -qq awscliv2.zip
    !sudo ./aws/install --update

    # upload
    output_name = MODEL_OUTPUT_PATH.split("/")[-1]
    os.system(
        f"aws s3 cp --recursive {MODEL_OUTPUT_PATH} s3://{COMPETITION_NAME}/trained_model/{output_name}"
    )

In [47]:
# ダウンロード（参考）
# !sudo rm /usr/bin/aws
# !sudo rm /usr/bin/aws_completer
# !sudo rm -rf /usr/local/aws-cli

# !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
# !unzip -o -qq awscliv2.zip
# !sudo ./aws/install --update

# !aws s3 cp --recursive s3://automated-essay-scoring/trained_model/e005-regression /notebooks/automated_essay_scoring/trained_models/e005-regression

# Kaggle Datasetへのupload

In [ ]:
import os

os.system("mkdir -p ~/.kaggle/")
os.system(f"cp {ENV_PATH}/kaggle.json ~/.kaggle/")
os.system("chmod 600 ~/.kaggle/kaggle.json")

In [ ]:
if not DEBUG and UPLOAD_DATA_TO_KAGGLE:
    import os
    import json

    from kaggle.api.kaggle_api_extended import KaggleApi

    def dataset_create_new(dataset_name: str, upload_dir: str):
        dataset_metadata = {}
        dataset_metadata["id"] = f"sinchir0/{dataset_name}"
        dataset_metadata["licenses"] = [{"name": "CC0-1.0"}]
        dataset_metadata["title"] = dataset_name
        with open(os.path.join(upload_dir, "dataset-metadata.json"), "w") as f:
            json.dump(dataset_metadata, f, indent=4)
        api = KaggleApi()
        api.authenticate()
        api.dataset_create_new(folder=upload_dir, convert_to_csv=False, dir_mode="tar")

    print(f"Create Dataset name:{DATASET_NAME}, output_dir:{MODEL_OUTPUT_PATH}")
    dataset_create_new(dataset_name=DATASET_NAME, upload_dir=MODEL_OUTPUT_PATH)

# ローカルからのデータの削除

In [50]:
# if not DEBUG and REMOVE_LOCAL_FILE:
#     # ローカルからは削除
#     os.system(f"rm -rf {MODEL_OUTPUT_PATH}")

In [51]:
if WANDB:
    wandb.finish()

In [52]:
print("finish Notebook!")